In [7]:
import pandas as pd
import pandapower as pp
import pandapower.plotting as pplt
import pandapower.topology as top
import networkx as nx
import plotly as plot
import pandapower.plotting.plotly as pplotly

import plotly.graph_objects as go
import numpy as np
from pandapower.plotting.plotly import get_plotly_color_palette


from create_net import createLVnet, createMVnet

from dash import Dash, html, dcc, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px

import plotly.graph_objects as go
import networkx as nx
import dash_cytoscape as cyto



import pandas as pd
import pandapower as pp
import pandapower.plotting as pplt
import pandapower.topology as top
import networkx as nx
import plotly as plot
import pandapower.plotting.plotly as pplotly

import plotly.graph_objects as go
import numpy as np
from pandapower.plotting.plotly import get_plotly_color_palette
from create_net import createLVnet, createMVnet

from numpy import floor

In [ ]:
##### creating MV net #####
pathMV = 'data/MV_benchmarks/'
netMV = createMVnet(pathMV+'bus33.json')

pp.runpp(netMV)

print(netMV.bus.head())

In [ ]:
lc = pplotly.create_line_trace(netMV,netMV.line[netMV.line.in_service == True].index, width=1.5, color='black')
bc = pplotly.create_bus_trace(netMV, netMV.bus.index, size=5, color="orange", infofunc=pd.Series(index=netMV.bus.index, data = 'Bus ' + netMV.bus.index.astype(str) + '<br>' + netMV.bus.vn_kv.astype(str) + ' kV'), trace_name='buses')
sc = pplotly.create_bus_trace(netMV, netMV.ext_grid.bus.values, size=5,patch_type="square", color='blue', trace_name='external connection')
# pplt.simple_plotly(netMV)
figMV = pplotly.draw_traces(lc + bc + sc, figsize=1, aspectratio=(8,6))


In [ ]:
figMV.write_html('figMV.html')

In [8]:

##### creating LV net #####
pathLV = 'data/spanish_LV_net/'

bus_df = pd.read_excel(pathLV+'bus_df.xlsx')
bus_df= bus_df.drop(columns={'Unnamed: 0'})

ext_grid_df = pd.read_excel(pathLV+'ext_grid_df.xlsx')

line_df = pd.read_excel(pathLV+'line_df.xlsx')
line_df = line_df.drop(columns={'Unnamed: 0'})


load_per_bus_df = pd.read_excel(pathLV+'load_per_bus_df.xlsx')

netLV = createLVnet(bus_df, line_df, ext_grid_df, load_per_bus_df)

pp.runpp(netLV)


In [12]:

pandanet = netLV
green_to_red = ['#00FF00','#11FF00','#22FF00','#33FF00','#44FF00','#55FF00','#66FF00','#77FF00','#88FF00','#99FF00',
                '#AAFF00','#BBFF00','#CCFF00','#DDFF00','#EEFF00','#FFFF00','#FFEE00','#FFDD00','#FFCC00','#FFAA00','#FF9900','#FF8800',
                '#FF7700','#FF6600','#FF5500','#FF4400','#FF3300','#FF2200','#FF1100','#FF0000']
def generate_stylesheet(bus_size,line_size):
    all_styles = [{
                    'selector': '.ext_grid',
                    'style': {
                        'background-color': 'yellow',
                        'shape': 'diamond',
                        'height': bus_size,
                        'width' : bus_size,
                        'border-width': 0.2,
                        'border-color': 'black',
                        }
                },
                {
                    'selector': '.buswhite',
                    'style': {
                        'background-color': 'white',
                        'width' : bus_size,
                        'height' : bus_size,
                        'border-width': 0.2,
                        'border-color': 'black',
                        }
                },
                {
                    'selector': '.line-overloaded',
                    'style':{
                        'line-color': green_to_red[29],
                        'width': line_size*2}
                },
                {
                    'selector': '.line-loaded-50-100',
                    'style':{
                        'line-color': green_to_red[14],
                        'width': line_size*1.5}
                },
                {
                    'selector': '.line-loaded-0-50',
                    'style':{
                        'line-color': green_to_red[0],
                        'width': line_size}
                },
                {
                    'selector': '.line-black',
                    'style':{
                        'line-color': 'black',
                        'width': line_size}
                },
                ]
    for hexcode in green_to_red:
        all_styles.append({
                    'selector': '.' + hexcode,
                    'style': {
                        'background-color': hexcode,
                        'width' : bus_size,
                        'height' : bus_size,
                        'border-width': 0.2,
                        'border-color': 'black',
                        }
                },)


    return all_styles

a = generate_stylesheet(2.0,1.0)
print(a)

[{'selector': '.ext_grid', 'style': {'background-color': 'yellow', 'shape': 'diamond', 'height': 2.0, 'width': 2.0, 'border-width': 0.2, 'border-color': 'black'}}, {'selector': '.buswhite', 'style': {'background-color': 'white', 'width': 2.0, 'height': 2.0, 'border-width': 0.2, 'border-color': 'black'}}, {'selector': '.line-overloaded', 'style': {'line-color': '#FF0000', 'width': 2.0}}, {'selector': '.line-loaded-50-100', 'style': {'line-color': '#EEFF00', 'width': 1.5}}, {'selector': '.line-loaded-0-50', 'style': {'line-color': '#00FF00', 'width': 1.0}}, {'selector': '.line-black', 'style': {'line-color': 'black', 'width': 1.0}}, {'selector': '.#00FF00', 'style': {'background-color': '#00FF00', 'width': 2.0, 'height': 2.0, 'border-width': 0.2, 'border-color': 'black'}}, {'selector': '.#11FF00', 'style': {'background-color': '#11FF00', 'width': 2.0, 'height': 2.0, 'border-width': 0.2, 'border-color': 'black'}}, {'selector': '.#22FF00', 'style': {'background-color': '#22FF00', 'width': 

In [206]:
from turtle import color
from numpy import floor


vlevels = pandanet.res_bus.vm_pu.values
gran = 6
maxgran = 30
diff = (1-vlevels)/0.05*gran
classis = floor(diff)
colorindex = classis*maxgran/gran
colorindex.astype(int)
print(colorindex)



[0. 5. 5. ... 0. 0. 0.]


In [ ]:
# lc = pplotly.create_line_trace(netLV,netLV.line.index, width=1.5, color='black')
# bc = pplotly.create_bus_trace(netLV, netLV.bus.index, size=2, color="orange", infofunc=pd.Series(index=netLV.bus.index, data='Bus '+ netLV.bus.index.astype(str) + '<br>' + netLV.bus.vn_kv.astype(str) + ' kV'))
# sc = pplotly.create_bus_trace(netLV, netLV.ext_grid.bus.values, size=2,patch_type="square", color='blue')
# figLV = pplotly.draw_traces(lc + bc + sc, figsize=1, aspectratio=(8,6))